In [1]:
import os
import pandas as pd
import numpy as np
from google.cloud import language_v1
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = "https://www.youtube.com/watch?v=txhlr7IUVLU"

In [3]:
data=[]

with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
    wait = WebDriverWait(driver,15)
    driver.get(url)

    for item in range(20): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(15)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

In [4]:
comment_df = pd.DataFrame(data, columns=['comment'])
comment_df = comment_df.drop(comment_df.index[0])

In [5]:
def sample_classify_text(text_content):
    try:
        client = language_v1.LanguageServiceClient()

        type_ = language_v1.Document.Type.PLAIN_TEXT

        language = "en"
        document = {"content": text_content, "type_": type_, "language": language}

        response = client.classify_text(request = {'document': document})

        for category in response.categories:
            print(u"Category name: {}".format(category.name))
            print(u"Confidence: {}".format(category.confidence))

        return category.name
    except:
        return "Less Relevant"

In [6]:
def repeat_until_twenty(comment):
    if (len(comment.split()) > 24):
        return comment
    else:
        while (len(comment.split()) < 25):
            comment = comment + ' ' + comment
    return comment

In [7]:
comment_df['comment'] = comment_df['comment'].astype(str)
comment_df['comment'].replace('', np.nan, inplace=True)
comment_df = comment_df.dropna()
comment_df['rep_comment'] = comment_df['comment'].apply(lambda x: repeat_until_twenty(x))

In [8]:
num_splits = round(len(comment_df)/500)+1
df_list = np.array_split(comment_df,num_splits)

In [9]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\mazon\jupyter-projects\personal-302809-c46a155d3fdf.json"

In [10]:
for df in df_list:
    df['classification'] = df['comment'].apply(lambda x: sample_classify_text(x))
    time.sleep(60)

Category name: /Arts & Entertainment
Confidence: 0.5899999737739563
Category name: /Arts & Entertainment
Confidence: 0.800000011920929
Category name: /Arts & Entertainment
Confidence: 0.6399999856948853
Category name: /Arts & Entertainment
Confidence: 0.5899999737739563
Category name: /Arts & Entertainment
Confidence: 0.699999988079071
Category name: /Arts & Entertainment
Confidence: 0.5699999928474426
Category name: /Arts & Entertainment
Confidence: 0.5199999809265137
Category name: /Food & Drink
Confidence: 0.6100000143051147
Category name: /Food & Drink/Food
Confidence: 0.5600000023841858
Category name: /Arts & Entertainment
Confidence: 0.5099999904632568
Category name: /Arts & Entertainment
Confidence: 0.6000000238418579
Category name: /Arts & Entertainment
Confidence: 0.8199999928474426
Category name: /News
Confidence: 0.5199999809265137
Category name: /Arts & Entertainment
Confidence: 0.9300000071525574
Category name: /Arts & Entertainment
Confidence: 0.5899999737739563
Category 

In [11]:
final_df = pd.concat(df_list)

In [12]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)  

<ipython-input-12-fe74e6e0f787>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [17]:
final_df = final_df.drop(columns = ['rep_comment'])

In [20]:
final_df.head()

,comment,classification
1,Taken from MMA Show #84 w/Brendan Schaub: \nhttps://youtu.be/Frw-IBgu-Qw,failed
2,"Sean: ""come on hots ones""\nJoe: ""no""\nSean: ""apparently fear is a factor for you""",/Arts & Entertainment
3,Hot ones interviewer is funny and doesn’t ask stupid repeatable questions. He’s does his homework. Rogan needs to get him on. That’d \nbe killer.,/Arts & Entertainment
4,"Brendan: ""I love Wings""\nJoe: ""Wings are overrated""\nBrendan: ""100%""",failed
5,If Sean Evans would let Joe put hot sauce on Elk loin he'd be there tomorrow.,failed


In [18]:
final_df.to_json("test.json")